In [1]:
import os
from glob import glob
import numpy as np
import sys
import pydicom
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

# from FindPoints.detect_points import detect_spheres_from_dicom
from detect_balls import detect_spheres_from_dicom
from cal_math import rigidTransform3D, transform3d, match_balls_by_geometry, Q2M
from plan_rom_data import plan_ball, plan_robot, plan_body, rom_body, rom_robot

[Taichi] version 1.7.3, llvm 15.0.1, commit 5ec301be, win, python 3.8.10
[Taichi] Starting on arch=x64


In [2]:
# 针尖法向量计算
rom_plane_robot_mat, _=rigidTransform3D(plan_robot[:4].T, rom_robot.T)  # 模型到rom的变换矩阵
# print(transform3d(plan_robot[:4], rom_plane_robot_mat))
# print(np.linalg.norm(transform3d(plan_robot[:4],rom_plane_robot_mat)-rom_robot,axis=1))  # 光球模型在rom下的位置和误差
r_pos = transform3d(plan_robot[4:],rom_plane_robot_mat)
r_plan_dis = r_pos[0] - r_pos[1]
tail_x = r_plan_dis / np.linalg.norm(r_plan_dis)
tail_p = r_pos[0]
tail_x, tail_p

(array([-0.6468745 , -0.30262412,  0.69998002]),
 array([52.04973873, -3.54322393,  7.67786135]))

In [3]:
def find_ct_ball(dicom_folder):
    dicom_paths = glob(os.path.join(dicom_folder, "*.dcm"))
    dicom_files = [pydicom.dcmread(path) for path in dicom_paths] 
    def get_z_position(dcm):
        if hasattr(dcm, 'SliceLocation'):
            # print("paixun 1")
            return float(dcm.SliceLocation)  # 优先使用 SliceLocation
        elif hasattr(dcm, 'ImagePositionPatient'):
            # print("paixun 2")
            return float(dcm.ImagePositionPatient[2])  # 否则使用 ImagePositionPatient 的 z 分量
        else:
            # print("paixun 3")
            return 0  # 如果没有 z 信息，默认 0
    dicom_files.sort(key=get_z_position)
    print(type(dicom_files))
    slices=dicom_files
    data = np.stack([s.pixel_array for s in slices])
    pixel_spacing = slices[0].PixelSpacing
    if hasattr(slices[0], 'SpacingBetweenSlices'):
        thickness = slices[0].SpacingBetweenSlices
    else:
        thickness = abs(slices[1].ImagePositionPatient[2] - slices[0].ImagePositionPatient[2])
    # print("体厚度=", thickness, pixel_spacing)
    #数据处理
    pos_ori = slices[-1].ImagePositionPatient
    data = np.flip(data, axis=0)
    axis = slices[0].ImageOrientationPatient
    print("PatientID:",slices[0].PatientID)
    #find points
    # print(f"坐标系={axis}")
    row_cosines = np.array(axis[0:3])
    col_cosines = np.array(axis[3:6])

    spheres = detect_spheres_from_dicom(data=data, pixel_spacing=pixel_spacing, thickness=thickness, pos_ori=pos_ori,
                                        r_range=0.75, rmax=5, rlen=10, output_points=15)
    spheres[:,0]=-pos_ori[0]+spheres[:,0] * pixel_spacing[0]*row_cosines[0]*-1
    spheres[:,1]=-pos_ori[1]+spheres[:,1] * pixel_spacing[1]*col_cosines[1]*-1
    spheres[:,2]=pos_ori[2]-spheres[:,2] * thickness
    return spheres


In [4]:
#规划
dicom_folder =  "C:/Users/YangLiangZhu/Desktop/泰州精度验证/Tai0606/9/dicom_data9_gui"
# dicom_folder = 'C:/Users/YangLiangZhu/Downloads/CT/CT/手术/MP25080/CT-MP25080-250606-6/S8010'
ct_all_ball = find_ct_ball(dicom_folder)
matched_points, _, _ = match_balls_by_geometry(plan_ball, ct_all_ball[:, :3], verbose=True)
ct_ball = matched_points[:, :3]
# ct_ball =  np.array([[  30.329681,    29.34164,   -399.524    ],
#  [  30.329681,    23.579922 , -373.524    ],
#  [  23.99179 ,    16.66586  , -336.524    ],
#  [   1.5210876 ,  18.970547 , -341.524    ]])
ct_plan_ball_mat, _ = rigidTransform3D(plan_ball.T, ct_ball.T)
ct_plan_ball_body = transform3d(plan_body, ct_plan_ball_mat)
T0, _ = rigidTransform3D(rom_body.T, ct_plan_ball_body.T)
ct_plan_ball_body


<class 'list'>
PatientID: CT-MP25080-250606-8
最佳匹配索引组合： [0, 1, 2, 3]
对应的检测点（已按实际顺序匹配）：
实际球 1 ⟶ 检测点 [  38.265625    8.596497 -278.89502 ]
实际球 2 ⟶ 检测点 [  39.71875    13.440247 -252.895   ]
实际球 3 ⟶ 检测点 [  33.90625    18.283997 -215.895   ]
实际球 4 ⟶ 检测点 [  12.59375    11.987122 -218.895   ]
总距离误差： 1.7967411970742315


array([[  37.61808863,    1.01142616, -300.47050752],
       [  43.16321965,   -2.81717284, -261.34451422],
       [  41.69762688,   12.25644033, -241.67503981],
       [  26.71688263,    1.82548005, -214.73254106]])

In [5]:
# 验证
ver_dicom_folder = "C:/Users/YangLiangZhu/Desktop/泰州精度验证/Tai0606/9/dicom_data9_yan"
# ver_dicom_folder = 'C:/Users/YangLiangZhu/Downloads/CT/CT/手术/MP25080/CT-MP25080-250606-7/S9010'

# print(time.time() - ts)
ver_ct_all_ball=find_ct_ball(ver_dicom_folder)

print(ver_ct_all_ball[:5, :3])
ver_matched_points, _, _ = match_balls_by_geometry(plan_ball, ver_ct_all_ball[:, :3],                                                               verbose=True)
ver_ct_ball = ver_matched_points[:, :3]
ver_ct_plan_ball_mat, _ = rigidTransform3D(plan_ball.T, ver_ct_ball.T)
# print("模型到CT坐标的刚性变换矩阵", ct_plan_ball_mat)

ver_ct_plan_ball_body = transform3d(plan_body, ver_ct_plan_ball_mat)
# print("ct下光球位置", ct_plan_ball_body)

ver_T0, _ = rigidTransform3D(rom_body.T, ver_ct_plan_ball_body.T)
ver_ct_plan_ball_body

<class 'list'>
PatientID: CT-MP25080-250606-9
[[  38.265625    8.596497 -278.89502 ]
 [  40.203125   13.440247 -252.895   ]
 [  12.109375   11.987122 -218.895   ]
 [  33.421875   18.283997 -215.895   ]
 [  65.875     109.3465   -320.89502 ]]
最佳匹配索引组合： [0, 1, 3, 2]
对应的检测点（已按实际顺序匹配）：
实际球 1 ⟶ 检测点 [  38.265625    8.596497 -278.89502 ]
实际球 2 ⟶ 检测点 [  40.203125   13.440247 -252.895   ]
实际球 3 ⟶ 检测点 [  33.421875   18.283997 -215.895   ]
实际球 4 ⟶ 检测点 [  12.109375   11.987122 -218.895   ]
总距离误差： 1.485317788634671


array([[  38.01585394,    1.0292275 , -300.41453563],
       [  43.18989697,   -2.83248915, -261.24099324],
       [  41.57456484,   12.23361805, -241.57749619],
       [  26.32234411,    1.82046972, -214.78081777]])

In [6]:
# CT里手点针尖的位置
ct_find_needle_tips = {1: np.array([8.637, 95.083, -348.771]), 
                       2: np.array([8.597, 94.855, -348.908]),
                       3: np.array([-24.785, 89.418, -348.672]),
                       4: np.array([-30.119, 101.916, -328.733]),
                       5: np.array([-34.021, 114.390, -424.759]),
                       6: np.array([-33.617, 114.151, -424.865]),
                       7: np.array([-1.510, 94.408, -403.805]),
                       8: np.array([-33.602, 91.015, -465.719]),
                       9: np.array([-43.074, 82.129, -467.753]),
                       10: np.array([-4.252, 76.807, -457.985])}

ct_find_needle_tails = {1: np.array([30.467, 7.622, -353.911]),
                        2: np.array([30.553, 7.436, -353.945]),
                        3: np.array([-39.718, 0.258, -345.846]),
                        4: np.array([-60.334, -85.350, -353.023]),
                        5: np.array([-104.274, -61.568, -421.010]),
                        6: np.array([-104.550, -62.330, -420.792]),
                        7: np.array([119.393, 21.571, -427.332]),
                        8: np.array([-84.018, -78.802, -457.905]),
                        9: np.array([-122.632, -50.054, -456.216]),
                        10: np.array([101.384, -51.280, -408.672])}

In [7]:
# REDIS数据
pose_to_ct = {1: np.array([[-1.1102230246251565e-16,
                            92.31198298007996,
                            -353.3949890136718],
                           [36.85393225106759,
                            -25.890258046204025,
                            -350.69835982456937]]),
              2: np.array([[],  # 没做
                           []]),
              3: np.array([
                  [
                      -28.31460648557631,
                      94.87265549658136,
                      -354.394989013672],
                  [
                      -45.39325801655887,
                      -35.91385453630318,
                      -349.00173063546674
                  ]]),
              4: np.array([
                  [
                      -32.8089884674138,
                      103.86141907272466,
                      -350.8444272118558
                  ],
                  [
                      -53.483145583866374,
                      -29.62172003300288,
                      -350.39498901367176
                  ]]),
              5: np.array([
                  [
                      -34.15730312009486,
                      102.06366635749596,
                      -431.788247391877
                  ],
                  [
                      -67.86516792574642,
                      1.8389524834985522,
                      -427.2938654100393
                  ]
              ]),
              6: np.array([  # 没做
                  [

                  ],
                  [

                  ]]
              ),
              7: np.array([[
                  -8,
                  107.0074863243748,
                  -406.57477513461515
              ],
                  [
                      74.24719040326252,
                      43.63670311256479,
                      -418.26016828739273
                  ]
              ]),
              8: np.array([[
                  -25.775280720085007,
                  103.41198089391746,
                  -459.60848252029757
              ],
                  [
                      -70.26966234027638,
                      -33.6666636422674,
                      -465.45117909668653
                  ]]),
              9: np.array([[-33.25842666559757,
                            92.1760264237384,
                            -465.394989013672],
                           [-129.43820088315445,
                            -26.475652781352707,
                            -460.9006070318342]]),
              10: np.array([[-4.943820180021258,
                             86.78276827805243,
                             -450.394989013672],
                            [73.70786450213518,
                             -22.43070917208827,
                             -450.39498901367176]])
              }

robot_after = {1: np.array([-163.148193359375,
                            212.99407958984375,
                            -1651.0927734375,
                            0.8833289742469788,
                            -0.3302684724330902,
                            -0.32147645950317383,
                            -0.08547268807888031]),
               2: np.array([-130.24606323242188,
                            218.477783203125,
                            -1527.9007568359375,
                            0.8530240058898926,
                            -0.5010212063789368,
                            -0.055626656860113144,
                            0.13503168523311615]),
               3: np.array([-161.05067443847656,
                            187.23045349121094,
                            -1652.1734619140625,
                            0.871877908706665,
                            -0.34895244240760803,
                            -0.333023339509964,
                            -0.08459670841693878]),
               4: np.array([-208.33087158203125,
                            77.00535583496094,
                            -1611.321044921875,
                            0.9560052752494812,
                            -0.27341169118881226,
                            -0.10555561631917953,
                            0.012565813958644867]),
               5: np.array([
                   -140.6206817626953,
                   255.92921447753906,
                   -1559.7490234375,
                   0.820655107498169,
                   -0.5669674277305603,
                   -0.05909746140241623,
                   -0.039758335798978806
               ],
               )
               }

spine_after = {1: np.array([-33.34297561645508,
                            43.048763275146484,
                            -1528.234619140625,
                            0.9649072885513306,
                            -0.06590159237384796,
                            0.22094786167144775,
                            0.12567009031772614]),
               2: np.array([-31.724733352661133,
                            42.38328170776367,
                            -1529.2227783203125,
                            0.9648662805557251,
                            -0.06486830860376358,
                            0.22007229924201965,
                            0.12803645431995392]),
               3: np.array([-31.7352237701416,
                            45.380958557128906,
                            -1530.553955078125,
                            0.9648716449737549,
                            -0.06484962999820709,
                            0.2196609377861023,
                            0.1287100613117218]),
               4: np.array([-32.76726531982422,
                            42.52442169189453,
                            -1529.698974609375,
                            0.965252161026001,
                            -0.06576952338218689,
                            0.21870079636573792,
                            0.12701416015625]),
               5: np.array([
                   -34.308326721191406,
                   29.772933959960938,
                   -1530.30810546875,
                   0.9653386473655701,
                   -0.07120106369256973,
                   0.21724843978881836,
                   0.12591595947742462
               ]
               )
               }

robot_before = {1: np.array([-135.85165405273438,
                             186.60446166992188,
                             -1778.3310546875,
                             0.9219368696212769,
                             -0.3848118185997009,
                             -0.04257398843765259,
                             0.011819192208349705]),
                2: np.array([]),
                3: np.array([
                    -173.91546630859375,
                    168.3963623046875,
                    -1873.732177734375,
                    0.8832840919494629,
                    -0.3917657136917114,
                    -0.2552027702331543,
                    -0.034646496176719666
                ]),
                4: np.array([23.689552307128906,
                            155.73431396484375,
                            -1739.515869140625,
                            0.7771176099777222,
                            0.0032706281635910273,
                            0.6293390393257141,
                            -0.00315357674844563
                    ]),
                5: np.array([-151.03428649902344,
                             91.60417175292969,
                             -1918.1328125,
                             0.8664283752441406,
                             -0.32389163970947266,
                             -0.36881938576698303,
                             -0.0914786085486412]),
                6: np.array([]),
                7: np.array([]),
                8: np.array([-165.2660369873047,
                             48.537715911865234,
                             -1889.4990234375,
                             0.9159523844718933,
                             -0.19436857104301453,
                             -0.3505324423313141,
                             -0.019471364095807076]),
                9: np.array([          -223.3853302001953,
        70.27033233642578,
        -1932.027099609375,
        0.881690502166748,
        -0.12311463803052902,
        -0.4472082555294037,
        -0.0864257961511612]),
                10: np.array([-152.63983154296875,
                              -210.029052734375,
                              -1723.353515625,
                              0.8849340081214905,
                              -0.44701582193374634,
                              0.12473800033330917,
                              -0.03884631022810936])
                }

spine_before = {1: np.array([-29.990947723388672,
                             -12.837776184082031,
                             -1784.919189453125,
                             0.959116518497467,
                             -0.07575679570436478,
                             0.23862174153327942,
                             0.13196974992752075]),
                2: np.array([]),
                3: np.array([
                    23.391864776611328,
                    150.80743408203125,
                    -1738.9505615234375,
                    0.778316080570221,
                    0.0007537917117588222,
                    0.627859890460968,
                    -0.0039338478818535805]),
                4: np.array([]),
                5: np.array([24.600757598876953,
                             152.58651733398438,
                             -1739.0601806640625,
                             0.7766873240470886,
                             0.0038468502461910248,
                             0.6298716068267822,
                             -0.00196087802760303]),
                6: np.array([]),
                7: np.array([]),
                8: np.array([-62.59754943847656,
                             235.97930908203125,
                             -1797.99658203125,
                             0.9277182817459106,
                             -0.06053275614976883,
                             0.35370245575904846,
                             -0.10280656069517136]),
                9: np.array([        -112.70799255371094,
        251.69407653808594,
        -1792.8687744140625,
        0.9208774566650391,
        -0.058551568537950516,
        0.37116739153862,
        -0.103880703449249275]),
                10: np.array([-117.12763214111328,
                              -105.92213439941406,
                              -1823.0274658203125,
                              0.9465838074684143,
                              -0.2888542413711548,
                              -0.1169838085770607,
                              -0.08280784636735916])
                }


In [ ]:
# change1
spine = Q2M(spine_before[9][3:], spine_before[9][:3])  # 光球自身相对于NDI的刚性矩阵
robot = Q2M(robot_before[9][3:], robot_before[9][:3])  # 机器人光球相对于NDI的刚性矩阵
spine, robot
robot_2_spine = np.linalg.inv(spine) @ robot
robot_2_spine

array([[-1.22613984e-01,  5.42221149e-02, -9.90972135e-01,
         5.82380944e+01],
       [-1.04364609e-01,  9.92265779e-01,  6.72060366e-02,
        -1.66891165e+02],
       [ 9.86951792e-01,  1.11662819e-01, -1.16006790e-01,
        -1.82441089e+02],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

In [9]:
# change2
ver_ct_ball_mat, _ = rigidTransform3D(ct_ball.T, ver_ct_ball.T)
ver_ct_ball_hope_needle = transform3d(pose_to_ct[9], ver_ct_ball_mat)  # 规划里的到验证ct里针尖的位置
ver_t0_needle=transform3d(pose_to_ct[9], np.linalg.inv(T0)@ver_T0)
ver_ct_ball_hope_needle, ver_ct_ball_hope_needle[0], ver_t0_needle[0]

(array([[ -31.14481914,   92.41878311, -465.94022367],
        [-127.60007204,  -26.04140501, -462.39938346]]),
 array([ -31.14481914,   92.41878311, -465.94022367]),
 array([ -31.45261901,   88.33828924, -465.91010426]))

In [10]:
# change3  规划转到验证里的和ct的
"规划和CT针尖距离：", ver_ct_ball_hope_needle[0]- ct_find_needle_tips[9] ,np.linalg.norm(ver_ct_ball_hope_needle[0] - ct_find_needle_tips[9])

('规划和CT针尖距离：',
 array([11.92918086, 10.28978311,  1.81277633]),
 15.857841927285023)

In [11]:
# change4 希望扎针 规划
hope_X_dis = pose_to_ct[9][1] - pose_to_ct[9][0]
hope_X_dis, hope_X_dis/np.linalg.norm(hope_X_dis)

(array([ -96.17977422, -118.65167921,    4.49438198]),
 array([-0.62943356, -0.77649744,  0.02941278]))

In [ ]:
#验证robot扎针, 实际扎针
ca_robot_x = robot[:3,:3]@np.array(tail_x)  # 变换为NDI下tail_x的方向向量
ct_robot_x = (T0@np.linalg.inv(spine)@robot)[:3,:3]@np.array(tail_x)  # 变换为CT下tail_x的向量
ct_robot_p = transform3d(np.array(tail_p).reshape(1,3),T0@np.linalg.inv(spine)@robot)
ct_robot_p2 = ct_robot_p - ct_robot_x * 140
ct_robot_x, ct_robot_p, ct_robot_p2

(array([-0.5352357 , -0.844698  ,  0.00283612]),
 array([[-113.48447521,  -27.690796  , -463.63747503]]),
 array([[ -38.55147745,   90.56692338, -464.03453168]]))

In [13]:
from cal_math import calculate_angle
"规划和计算角度", calculate_angle(hope_X_dis, ct_robot_x)  # 规划和验证的

('规划和计算角度', 6.839057158691965)

In [14]:
"计算和规划针尖距离", ct_robot_p2 - ver_ct_ball_hope_needle[0] ,np.linalg.norm(ct_robot_p2 - ver_ct_ball_hope_needle[0])

('计算和规划针尖距离',
 array([[-7.40665831, -1.85185973,  1.90569199]]),
 7.868902959522763)

In [15]:
# 验证的和ct
"计算和CT针尖距离", ct_robot_p2 - ct_find_needle_tips[9] ,np.linalg.norm(ct_robot_p2 - ct_find_needle_tips[9])

('计算和CT针尖距离',
 array([[4.52252255, 8.43792338, 3.71846832]]),
 10.270285673588702)

In [16]:
# change5 CT 扎针 
hand_ct_X_dis = ct_find_needle_tails[9] - ct_find_needle_tips[9]
hand_ct_X_dis, hand_ct_X_dis / np.linalg.norm(hand_ct_X_dis)

(array([ -79.558, -132.183,   11.537]),
 array([-0.51424235, -0.85439674,  0.07457219]))

In [17]:
"规划和CT角度", calculate_angle(hand_ct_X_dis, hope_X_dis)  # ct的和规划的

('规划和CT角度', 8.384562909607853)

In [1]:
"计算和CT角度", calculate_angle(hand_ct_X_dis, ct_robot_x)  # ct的实际的

NameError: name 'calculate_angle' is not defined